In [41]:
import sys
import os
import datetime as dt
import time
from tqdm import tqdm
import pandas as pd
import torch
from torch import nn
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import precision_recall_fscore_support


# TODO: change path name
sys.path.append("/home/bchau/Math_156_temp/Final_Project/preprocessing/")
from preprocessing import EuroSATDataset

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
# TODO: change path name
# setting paths to EuroSAT data and preprocessing statistics
data_path = '/home/bchau/Math_156_temp/Final_Project/EuroSAT_RGB'
preprocessing_stats_path = '/home/bchau/Math_156_temp/Final_Project/preprocessing/preprocessing_stats.pkl'
checkpoint_path = '/home/bchau/Math_156_temp/Final_Project/checkpoints'

In [44]:
# getting eurosat dataset
eurosat = EuroSATDataset(data_path, preprocessing_stats_path, transform=True)
classes = eurosat.sorted_class_names

In [45]:
# splitting dataset into train, validation, and test
generator = torch.Generator().manual_seed(0)
train_val_set, test_set = random_split(eurosat, [0.8, 0.2], generator = generator)
train_set, val_set = random_split(train_val_set, [0.8, 0.2], generator = generator)

In [46]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3)
        self.pooling = nn.MaxPool2d(2, 2)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.conv(x) 
        x = self.pooling(x)
        x = self.batch_norm(x) 
        x = self.relu(x) 
        return x

In [47]:
class FullyConnectedBlock(nn.Module):
    def __init__(self, in_channels, out_channels, is_output=False):
        super().__init__()
        self.is_output = is_output
        self.conv = nn.Linear(in_channels, out_channels)
        if not self.is_output:
            self.batch_norm = nn.BatchNorm1d(out_channels)
            self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        if not self.is_output: 
            x = self.batch_norm(x)
            x = self.relu(x)
        return x

In [48]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_block1 = ConvBlock(3, 8)
        self.conv_block2 = ConvBlock(8, 12)
        self.conv_block3 = ConvBlock(12, 16)
        flatten_channels = 16 * 6 * 6
        self.fc_block1 = FullyConnectedBlock(flatten_channels, flatten_channels // 2)
        self.fc_block2 = FullyConnectedBlock(flatten_channels // 2, 10, is_output=True)
        self.conv_blocks = nn.Sequential(
            self.conv_block1, 
            self.conv_block2, 
            self.conv_block3
        )
        self.fc_blocks = nn.Sequential(
            self.fc_block1, 
            self.fc_block2
        )

    def forward(self, x):
        x = self.conv_blocks(x)
        #  flatten all dimensions except batch
        x = torch.flatten(x, 1) 
        x = self.fc_blocks(x)
        return x
    
test_model = Net()
test_img = torch.rand((10, 3, 64, 64))
test_model(test_img).shape

torch.Size([10, 10])

In [49]:
# Initializing model
model_name = 'cnn' # name of model (for checkpoint file name)
model = Net().to(device)

In [50]:
# TODO: setting hyperparameters
batch_size = 64
epochs = 20 
optimizer = torch.optim.SGD(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()

# containers for storing loss data and epoch times
train_loss = []
train_loss_idx = []
val_loss = []
val_loss_idx = []
epoch_times = []

# tracking when to checkpoint model
checkpoint_after_epochs = 5

In [51]:
# creating dataloaders
train_loader = DataLoader(train_set, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size = batch_size)

In [52]:
# Testing loss
for data in train_loader:
    imgs = data['image'].to(device)
    labels = data['land_use'].to(device)
    print(imgs.shape)
    print(imgs.dtype)
    print(labels.shape)
    print(labels.dtype)
    test_output = model(imgs)
    print(test_output.dtype)
    print(torch.nn.CrossEntropyLoss()(test_output, labels))
    break

torch.Size([64, 3, 64, 64])
torch.float32
torch.Size([64])
torch.int64
torch.float32
tensor(2.4901, device='cuda:0', grad_fn=<NllLossBackward0>)


In [53]:
def train_one_epoch(epoch_index, optimizer, loss_fn, train_loader, model, 
                    train_loss, train_loss_idx):
    running_loss = 0.
    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs = data['image'].to(device)
        labels = data['land_use'].to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item().detach()
        if i % 90 == 89:
            last_loss = running_loss / 90 # loss per batch
            timestamp = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
            print('{} batch {} loss: {}'.format(timestamp, i + 1, last_loss))
            train_loss_idx.append(epoch_index * len(train_loader) + i + 1)
            train_loss.append(last_loss)
            running_loss = 0.

    return last_loss

In [54]:
def save_model(epoch, optimizer, loss_fn, model, 
               train_loss, train_loss_idx, val_loss, val_loss_idx, status):
    model_path = os.path.join(checkpoint_path, f'{status}_{model_name}_e{epoch}.tar')
    result = {
        'epoch': epoch, 
        'optimizer_state_dict': optimizer.state_dict(),
        'loss_fn': loss_fn, 
        'model_state_dict': model.state_dict(),
        'train_loss': train_loss, 
        'train_loss_idx': train_loss_idx, 
        'val_loss': val_loss, 
        'val_loss_idx': val_loss_idx
    }
    timestamp = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
    print(f'{timestamp} Saving results at {checkpoint_path}')
    torch.save(result, model_path)

In [55]:
def load_model(checkpoint_path, model_type, optimizer_type=None):
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    model_type.load_state_dict(checkpoint['model_state_dict'])
    # returns all training information if optimizer is provided
    if optimizer_type:
        optimizer_type.load_state_dict(checkpoint['optimizer_state_dict'])
        model_epoch = checkpoint['epoch']
        loss_fn = checkpoint['loss_fn']
        train_loss = checkpoint['train_loss']
        train_loss_idx = checkpoint['train_loss_idx']
        val_loss = checkpoint['val_loss']
        val_loss_idx = checkpoint['val_loss_idx']
        return (model_epoch, optimizer_type, loss_fn, model_type, 
                train_loss, train_loss_idx, val_loss, val_loss_idx)
    # otherwise only return model
    return model_type

In [56]:
def train_model(epochs, optimizer, loss_fn, train_loader, model):
    best_vloss = torch.inf 
    for epoch in range(epochs):
        timestamp = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        print(f"{timestamp} Epoch {epoch}/{epochs}")

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        epoch_start_time = time.time()
        avg_loss = train_one_epoch(epoch, optimizer, loss_fn, train_loader, model, train_loss, train_loss_idx)
        epoch_end_time = time.time()
        epoch_times.append(epoch_end_time - epoch_start_time)

        timestamp = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        print(f"{timestamp} Finished training in {str(dt.timedelta(seconds = epoch_times[-1]))}")

        # Set the model to evaluation mode, disabling dropout and using population
        # statistics for batch normalization.
        model.eval()

        running_vloss = 0.0
        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            for i, vdata in enumerate(val_loader):
                vinputs = vdata['image'].to(device)
                vlabels = vdata['land_use'].to(device)
                voutputs = model(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                running_vloss += vloss.item().detach()

        avg_vloss = running_vloss / (i + 1)
        timestamp = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
        print('{} LOSS train {} valid {}'.format(timestamp, avg_loss, avg_vloss))

        # Log the validation running loss averaged per batch
        val_loss_idx.append(len(train_loader) * (epoch + 1))
        val_loss.append(avg_vloss)

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            timestamp = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
            print(f"{timestamp} New best validation loss: {best_vloss}")
            save_model(epoch + 1, optimizer, loss_fn, model, 
                       train_loss, train_loss_idx, val_loss, val_loss_idx, 'best')
        elif epoch % checkpoint_after_epochs == checkpoint_after_epochs - 1:
            save_model(epoch + 1, optimizer, loss_fn, model, 
                       train_loss, train_loss_idx, val_loss, val_loss_idx, 'latest')
        
        print('=================================')

    save_model(epoch + 1, optimizer, loss_fn, model, 
               train_loss, train_loss_idx, val_loss, val_loss_idx, 'latest')

In [34]:
train_model(epochs, optimizer, loss_fn, train_loader, model)

2025-03-09 17-29-09 Epoch 0/20


AttributeError: 'float' object has no attribute 'cpu'

In [75]:
def test_model(data_loader, model):
    all_labels = torch.zeros(len(data_loader.dataset))
    all_predictions = torch.zeros(len(data_loader.dataset))
    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader)):
            inputs = data['image'].to(device)
            labels = data['land_use'].cpu()
            outputs = model(inputs)
            predictions = torch.argmax(outputs, dim=1).cpu()
            start_idx = i * data_loader.batch_size
            if i != len(data_loader) - 1:
                end_idx = start_idx + data_loader.batch_size
            else:
                end_idx = start_idx + len(labels)
                assert(end_idx == len(data_loader.dataset))
            all_labels[start_idx:end_idx] = labels
            all_predictions[start_idx:end_idx] = predictions 

    metrics = pd.DataFrame(columns=['Precision', 'Recall', 'F1_Score'])
    
    micro_avg = precision_recall_fscore_support(all_labels, all_predictions, 
                                                average='micro', zero_division='warn')
    macro_avg = precision_recall_fscore_support(all_labels, all_predictions, 
                                                average='macro', zero_division='warn')
    accuracy = torch.sum(all_labels == all_predictions) / len(all_labels)
    metrics.loc['Micro Avg'] = micro_avg[:3]
    metrics.loc['Macro Avg'] = macro_avg[:3]
    metrics.loc['Accuracy'] = [accuracy.item(), None, None]

    return metrics

In [76]:
latest_model_path = os.path.join(checkpoint_path, 'first_run', 'latest_cnn_e19')
checkpoint_model = Net()
checkpoint_model = load_model(latest_model_path, checkpoint_model).to(device)

In [77]:
metrics = test_model(test_loader, checkpoint_model)

  0%|          | 0/85 [00:00<?, ?it/s]

100%|██████████| 85/85 [00:20<00:00,  4.17it/s]


In [78]:
metrics

,Precision,Recall,F1_Score
Micro Avg,0.774444,0.774444,0.774444
Macro Avg,0.765669,0.768775,0.764691
Accuracy,0.774444,NaN,NaN


In [ ]:
# TODO: grid search specific to CNN, googlenet, mobilenet